In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("hw7.ipynb")

# CPSC 330 - Applied Machine Learning 

## Homework 7: Word embeddings and topic modeling 
**Due date: See the [Calendar](https://htmlpreview.github.io/?https://github.com/UBC-CS/cpsc330/blob/master/docs/calendar.html).**

## Imports

In [2]:
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline

<br><br>

## Submission instructions
<hr>
rubric={points:2}

You will receive marks for correctly submitting this assignment. To submit this assignment, follow the instructions below:

- **You may work on this assignment in a group (group size <= 4) and submit your assignment as a group.** 
- Below are some instructions on working as a group.  
    - The maximum group size is 4. 
    - You can choose your own group members. 
    - Use group work as an opportunity to collaborate and learn new things from each other. 
    - Be respectful to each other and make sure you understand all the concepts in the assignment well. 
    - It's your responsibility to make sure that the assignment is submitted by one of the group members before the deadline. [Here](https://help.gradescope.com/article/m5qz2xsnjy-student-add-group-members) are some instructions on adding group members in Gradescope.  
- Upload the .ipynb file to Gradescope.
- **If the .ipynb file is too big or doesn't render on Gradescope for some reason, also upload a pdf or html in addition to the .ipynb.** 
- Make sure that your plots/output are rendered properly in Gradescope.

<br><br><br><br>

## Exercise 1:  Exploring pre-trained word embeddings <a name="1"></a>
<hr>

In lecture 18, we talked about natural language processing (NLP). Using pre-trained word embeddings is very common in NLP. It has been shown that pre-trained word embeddings work well on a variety of text classification tasks. These embeddings are created by training a model like Word2Vec on a huge corpus of text such as a dump of Wikipedia or a dump of the web crawl. 

A number of pre-trained word embeddings are available out there. Some popular ones are: 

- [GloVe](https://nlp.stanford.edu/projects/glove/)
    * trained using [the GloVe algorithm](https://nlp.stanford.edu/pubs/glove.pdf) 
    * published by Stanford University 
- [fastText pre-trained embeddings for 294 languages](https://fasttext.cc/docs/en/pretrained-vectors.html) 
    * trained using the fastText algorithm
    * published by Facebook
    
In this exercise, you will be exploring GloVe Wikipedia pre-trained embeddings. The code below loads the word vectors trained on Wikipedia using an algorithm called Glove. You'll need `gensim` package in your cpsc330 conda environment to run the code below. 

```
> conda activate cpsc330
> conda install -c anaconda gensim
```

In [3]:
import gensim
import gensim.downloader

print(list(gensim.downloader.info()["models"].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [4]:
# This will take a while to run when you run it for the first time.
import gensim.downloader as api

glove_wiki_vectors = api.load("glove-wiki-gigaword-100")

In [5]:
len(glove_wiki_vectors)

400000

There are 400,000 word vectors in this pre-trained model. 

Now that we have GloVe Wiki vectors loaded in `glove_wiki_vectors`, let's explore the embeddings. 

<br><br>

<!-- BEGIN QUESTION -->

### 1.1 Word similarity using pre-trained embeddings
rubric={points:2}

**Your tasks:**

- Come up with a list of 4 words of your choice and find similar words to these words in `glove_wiki_vectors` embeddings. 

<div class="alert alert-warning">

Solution_1.1
    
</div>

In [6]:
glove_wiki_vectors.similar_by_word('bioinformatics')

[('computational', 0.6967540383338928),
 ('genomics', 0.6863164901733398),
 ('nanotechnology', 0.6814009547233582),
 ('neuroscience', 0.669538676738739),
 ('proteomics', 0.6518864631652832),
 ('nanoscience', 0.6322058439254761),
 ('informatics', 0.6133146286010742),
 ('biomedical', 0.6071567535400391),
 ('genomic', 0.6048443913459778),
 ('biology', 0.6036946177482605)]

In [7]:
glove_wiki_vectors.similar_by_word('varada')

[('zholtok', 0.6019389629364014),
 ('zhitnik', 0.6012267470359802),
 ('afinogenov', 0.5898696184158325),
 ('kozlov', 0.5801160931587219),
 ('gratton', 0.5793241858482361),
 ('smehlik', 0.5753229856491089),
 ('nkufo', 0.5738336443901062),
 ('krivokrasov', 0.560421884059906),
 ('sverkos', 0.5470700860023499),
 ('berezin', 0.5400629043579102)]

In [8]:
glove_wiki_vectors.similar_by_word('apples')

[('pears', 0.8060726523399353),
 ('mangoes', 0.7937251329421997),
 ('oranges', 0.7910456657409668),
 ('peaches', 0.7866174578666687),
 ('cherries', 0.7693213820457458),
 ('strawberries', 0.7389282584190369),
 ('melons', 0.7277671098709106),
 ('apricots', 0.7255706191062927),
 ('tomatoes', 0.7237654328346252),
 ('potatoes', 0.7182415723800659)]

In [9]:
glove_wiki_vectors.similar_by_word('apple')

[('microsoft', 0.7449405789375305),
 ('ibm', 0.6821643114089966),
 ('intel', 0.6778088212013245),
 ('software', 0.6775422692298889),
 ('dell', 0.6741442680358887),
 ('pc', 0.6678153276443481),
 ('macintosh', 0.66175377368927),
 ('iphone', 0.6595611572265625),
 ('ipod', 0.65346759557724),
 ('hewlett', 0.6516579985618591)]

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.2 Word similarity using pre-trained embeddings
rubric={points:2}

**Your tasks:**
1. Calculate cosine similarity for the following word pairs (`word_pairs`) using the [`similarity`](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) method of `glove_wiki_vectors`.

In [10]:
word_pairs = [
    ("coast", "shore"),
    ("clothes", "closet"),
    ("old", "new"),
    ("smart", "intelligent"),
    ("dog", "cat"),
    ("tree", "lawyer"),
]

<div class="alert alert-warning">

Solution_1.2
    
</div>

In [11]:
for pair in word_pairs:
    print('Consine similarity between "%s" and "%s"' % pair)
    print(glove_wiki_vectors.similarity(*pair), end = '\n\n')

Consine similarity between "coast" and "shore"
0.7000272

Consine similarity between "clothes" and "closet"
0.54627603

Consine similarity between "old" and "new"
0.6432488

Consine similarity between "smart" and "intelligent"
0.7552733

Consine similarity between "dog" and "cat"
0.87980753

Consine similarity between "tree" and "lawyer"
0.076719455



<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.3 Stereotypes and biases in embeddings
rubric={points:6}

Word vectors contain lots of useful information. But they also contain stereotypes and biases of the texts they were trained on. In the lecture, we saw an example of gender bias in Google News word embeddings. Here we are using pre-trained embeddings trained on Wikipedia data. 

**Your tasks:**

1. Explore whether there are any worrisome biases or stereotypes present in these embeddings or not by trying out at least 4 examples. You can use the following two methods or other methods of your choice to explore what kind of stereotypes and biases are encoded in these embeddings. 
    - the `analogy` function below which gives word analogies (an example shown below)
    - [similarity](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) or [distance](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=distance#gensim.models.keyedvectors.KeyedVectors.distances) methods (an example is shown below)   
2. Discuss your observations.

> Note that most of the recent embeddings are de-biased. But you might still observe some biases in them. Also, not all stereotypes present in pre-trained embeddings are necessarily bad. But you should be aware of them when you use them in your models. 

An example of using analogy to explore biases and stereotypes.  

In [12]:
def analogy(word1, word2, word3, model=glove_wiki_vectors):
    """
    Returns analogy word using the given model.

    Parameters
    --------------
    word1 : (str)
        word1 in the analogy relation
    word2 : (str)
        word2 in the analogy relation
    word3 : (str)
        word3 in the analogy relation
    model :
        word embedding model

    Returns
    ---------------
        pd.dataframe
    """
    print("%s : %s :: %s : ?" % (word1, word2, word3))
    sim_words = model.most_similar(positive=[word3, word2], negative=[word1])
    return pd.DataFrame(sim_words, columns=["Analogy word", "Score"])

In [13]:
analogy("man", "doctor", "woman")

man : doctor :: woman : ?


,Analogy word,Score
0,nurse,0.773523
1,physician,0.718943
2,doctors,0.682433
3,patient,0.675068
4,dentist,0.672603
5,pregnant,0.664246
6,medical,0.652045
7,nursing,0.645348
8,mother,0.639333
9,hospital,0.638750


An example of using similarity between words to explore biases and stereotypes.  

In [14]:
glove_wiki_vectors.similarity("white", "rich")

0.447236

In [15]:
glove_wiki_vectors.similarity("black", "rich")

0.51745194

<div class="alert alert-warning">

Solution_1.3
    
</div>

In [16]:
analogy('man', 'scientist', 'woman')

man : scientist :: woman : ?


,Analogy word,Score
0,researcher,0.777012
1,anthropologist,0.684751
2,sociologist,0.670103
3,psychologist,0.658697
4,physicist,0.657407
5,professor,0.650351
6,biologist,0.643093
7,geneticist,0.641517
8,biochemist,0.623625
9,expert,0.613421


In [17]:
analogy('woman', 'scientist', 'man')

woman : scientist :: man : ?


,Analogy word,Score
0,physicist,0.712791
1,expert,0.687481
2,biologist,0.637897
3,professor,0.631798
4,researcher,0.625697
5,geologist,0.592915
6,engineer,0.586461
7,astronomer,0.583323
8,science,0.578497
9,mathematician,0.576410


In [18]:
analogy('black', 'human', 'white')

black : human :: white : ?


,Analogy word,Score
0,rights,0.640116
1,animal,0.630374
2,administration,0.581599
3,progress,0.577176
4,present,0.567762
5,response,0.567199
6,scientific,0.566410
7,organization,0.564222
8,cannot,0.563442
9,environment,0.559916


In [19]:
analogy('white', 'human', 'black')

white : human :: black : ?


,Analogy word,Score
0,animal,0.693573
1,rights,0.659524
2,body,0.646985
3,society,0.624834
4,beings,0.622379
5,life,0.620003
6,humans,0.616726
7,animals,0.606307
8,child,0.605153
9,people,0.603449


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.4 Discussion
rubric={points:4}

**Your tasks:**
1. Based on your exploration above, comment on the overall quality of these pre-trained embeddings. 
2. In the lecture, we saw that our pre-trained word embedding model output an analogy that reinforced a gender stereotype. Give an example of how using such a model could cause harm in the real world.

<div class="alert alert-warning">

Solution_1.4
    
</div>

**Science, Man, and Woman**
The first two examples focused on the relationship between "woman" and "man" with the title of "scientist". The objective was to determine gender-based biases that may be present in embedding. The results may differ depending on gender or sex terms.

The algebraic expression for the first example is: $scientist - man + woman$. The objective was to remove the "man" component of the "scientist" vector, and add the "woman" vector, and observe the more similar words from then on. Interestingly, there are multiple social and life science words. When the algebraic expression is reversed, $scientist - woman + man$, mathematics, hard, and life sciences appear. 

There is an overlap between the two expressions: "professor", "biologist", "researcher", etc. However, the degree of similarity is different between the two lists.

**Human, White, and Black**
The next two examples focused on the relationship between "white" and "black" with the word "human". The objective was to determine racial biases present in the embedding.

The algebraic expression for the first example is: $human - black + white$. The resulting top 10 words were "rights", "animals", "progress", "present", "scientific", etc.

The algebraic expression for the second example is: $human - white + black$. There was an overlap with the above the previous example, such as "animal" and "rights". However, there was no mention of words that indicate progress, structure, or thought.

**Potential Harm**
A word, or concept that the word represents, may have multiple biases attached to it. Some biases may stem from generalizations that have present day merit, and other may be unfounded. 

Moreover, biases are dynamic. If the word embedding is not updated recurrently, the current biases that may hold valuable information may be missed. Perhaps more problematically, biases from the past may be captured in a word embedding causing individuals to be discriminated against with past and current perceptions. Debiasing of word embeddings may be more challenging for biases that are on the rise, since they may not be detectable. More systematic biases based on historical accounts and academic research may be easier to detect and mitigate.



<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.5 Classification with pre-trained embeddings 
rubric={points:8}

In lecture 18, we saw that you can conveniently get word vectors with `spaCy` with `en_core_web_md` model. In this exercise, you'll use word embeddings in multi-class text classification task. We will use [HappyDB](https://www.kaggle.com/ritresearch/happydb) corpus which contains about 100,000 happy moments classified into 7 categories: *affection, exercise, bonding, nature, leisure, achievement, enjoy_the_moment*. The data was crowd-sourced via [Amazon Mechanical Turk](https://www.mturk.com/). The ground truth label is not available for all examples, and in this lab, we'll only use the examples where ground truth is available (~15,000 examples). 

- Download the data from [here](https://www.kaggle.com/ritresearch/happydb).
- Unzip the file and copy it under data/ directory in this homework directory.

The code below reads the data CSV (assuming that it's present in the current directory as *cleaned_hm.csv*),  cleans it up a bit, and splits it into train and test splits. 

**Your tasks:**

1. Train logistic regression with bag-of-words features (`CountVectorizer`) and show [classification report](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html) on the test set. 
2. Train logistic regression with average embedding representation extracted using spaCy and show [classification report](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html) on the test set. (You can refer to lecture 18 notes for this.)  

In [20]:
df = pd.read_csv("data/cleaned_hm.csv", index_col=0)
sample_df = df.dropna()
sample_df.head()

,wid,reflection_period,original_hm,cleaned_hm,modified,num_sentence,ground_truth_category,predicted_category
hmid,,,,,,,,
27676,206,24h,We had a serious talk with some friends of our...,We had a serious talk with some friends of our...,True,2,bonding,bonding
27678,45,24h,I meditated last night.,I meditated last night.,True,1,leisure,leisure
27697,498,24h,My grandmother start to walk from the bed afte...,My grandmother start to walk from the bed afte...,True,1,affection,affection
27705,5732,24h,I picked my daughter up from the airport and w...,I picked my daughter up from the airport and w...,True,1,bonding,affection
27715,2272,24h,when i received flowers from my best friend,when i received flowers from my best friend,True,1,bonding,bonding


In [21]:
sample_df = sample_df.rename(
    columns={"cleaned_hm": "moment", "ground_truth_category": "target"}
)

In [22]:
train_df, test_df = train_test_split(sample_df, test_size=0.3, random_state=123)
X_train, y_train = train_df["moment"], train_df["target"]
X_test, y_test = test_df["moment"], test_df["target"]

You need `spacy` to run the code below. If it's not in your course conda environment, you need to install it. 

> conda install -c conda-forge spacy

You also need to download the following language model. 

> python -m spacy download en_core_web_md

In [23]:
import spacy

nlp = spacy.load("en_core_web_md")

<div class="alert alert-warning">

Solution_1.5
    
</div>

_Type your answer here, replacing this text._

In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report
from sklearn.pipeline import make_pipeline

In [25]:
pipe_lr = make_pipeline(
    CountVectorizer(lowercase = True, stop_words = 'english'),
    LogisticRegression(n_jobs = -1, max_iter = 1000)
)
pipe_lr.fit(X_train, y_train)
print(classification_report(y_test, pipe_lr.predict(X_test)))

                  precision    recall  f1-score   support

     achievement       0.79      0.87      0.83      1302
       affection       0.90      0.91      0.91      1423
         bonding       0.91      0.85      0.88       492
enjoy_the_moment       0.60      0.54      0.57       469
        exercise       0.91      0.57      0.70        74
         leisure       0.73      0.70      0.71       407
          nature       0.73      0.46      0.57        71

        accuracy                           0.82      4238
       macro avg       0.80      0.70      0.74      4238
    weighted avg       0.82      0.82      0.81      4238



In [26]:
print(classification_report(y_train, pipe_lr.predict(X_train)))

                  precision    recall  f1-score   support

     achievement       0.94      0.97      0.96      2974
       affection       0.97      0.99      0.98      3387
         bonding       0.98      0.96      0.97      1258
enjoy_the_moment       0.92      0.87      0.90      1045
        exercise       0.99      0.82      0.90       143
         leisure       0.93      0.90      0.92       899
          nature       0.98      0.94      0.96       181

        accuracy                           0.96      9887
       macro avg       0.96      0.92      0.94      9887
    weighted avg       0.96      0.96      0.96      9887



In [27]:
from sklearn.base import BaseEstimator, TransformerMixin

class Embedder(BaseEstimator, TransformerMixin):
    def __init__(self, embedding):
        self.embedding = embedding
        
    def fit(self, X, y = None):
        return self
    
    def transform(self, X, y = None):
        return np.vstack([text.vector for text in self.embedding.pipe(X)])
    
    
pipe_lr_emb = make_pipeline(
    Embedder(embedding = nlp),
    LogisticRegression(n_jobs = -1, max_iter = 5000)
)
pipe_lr_emb.fit(X_train, y_train)
print(classification_report(y_test, pipe_lr_emb.predict(X_test)))

                  precision    recall  f1-score   support

     achievement       0.81      0.83      0.82      1302
       affection       0.86      0.91      0.88      1423
         bonding       0.83      0.77      0.80       492
enjoy_the_moment       0.56      0.51      0.54       469
        exercise       0.68      0.76      0.72        74
         leisure       0.72      0.65      0.68       407
          nature       0.68      0.72      0.70        71

        accuracy                           0.79      4238
       macro avg       0.74      0.74      0.73      4238
    weighted avg       0.79      0.79      0.79      4238



In [28]:
print(classification_report(y_train, pipe_lr_emb.predict(X_train)))

                  precision    recall  f1-score   support

     achievement       0.83      0.87      0.85      2974
       affection       0.90      0.93      0.91      3387
         bonding       0.88      0.84      0.86      1258
enjoy_the_moment       0.70      0.63      0.66      1045
        exercise       0.95      0.94      0.95       143
         leisure       0.79      0.72      0.75       899
          nature       0.93      0.97      0.95       181

        accuracy                           0.85      9887
       macro avg       0.86      0.84      0.85      9887
    weighted avg       0.85      0.85      0.85      9887



<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.6 Discussion
rubric={points:6}

**Your tasks:**
1. Briefly explain the difference between using `CountVectorizer` vs. average-embedding approach for text classification.  
2. Which representation among these two would be more interpretable? Why?   
3. Are we using any transfer learning here? If yes, are you observing any benefits of transfer learning? Briefly discuss. 

<div class="alert alert-warning">

Solution_1.6
    
</div>

1. Both are methods of that attempt to map words in a document to a vector space, which allows us to use machine learning algorithms and statistical methods. `CountVectorizer` is a naive approach that relies on frequency of words per document. A average-embedding uses a pre-trained deep learning model that takes in a word and maps it into a pre-existing vector space. The position a word is mapped to when using average-embedding relies on its cosine similarity to other words present in that vector space.

2. In terms of explaining the methodology used in order to extract the features for modeling, `CountVectorizer` is more interpretable. The machine learning practioner has total control over the vectorization of the words, and the methodology is simpler to grasp for non-technical stakeholders. A word embedding requires minimal expertise in its utilization, but the embedding generation process is a technically involved process that may not be interpretable to technical and non-technical stakeholder.

3. Use a embedding is one form of transfer learning, since the vector space has already been generated. The embedding is simply asked to vectorize a word based on the corpus it has been trained on. A benefit that can be observed through the classification reports generated above is the decreased extent of overfitting when using the embedding. 

The training scores on the `CountVectorizer` data are greater than the training scores obtained from the embedding. The test scores between `CountVectorizer` and the embedding are similar, meaning that the gap between training and test scores is smaller using the embedding. The decreased gap between the train and test scores suggests to me that the embedding is not overfittig as much as the `CountVectorizer`.

<!-- END QUESTION -->

<br><br><br><br>

## Exercise 2: Topic modeling 

The overarching goal of topic modeling is understanding high-level themes in a large collection of texts in an unsupervised way. 

In this exercise you will explore topics in a subset of `scikit-learn`'s [20 newsgroups text dataset](https://scikit-learn.org/0.19/datasets/twenty_newsgroups.html) using `scikit-learn`'s `LatentDirichletAllocation` (LDA) model. 

Usually, topic modeling is used for discovering abstract "topics" that occur in a collection of documents when you do not know the actual topics present in the documents. But 20 newsgroups text dataset is labeled with categories (e.g., sports, hardware, religion), and you will be able to cross-check the topics discovered by your model with these available topics. 

The starter code below loads the train and test portion of the data and convert the train portion into a pandas DataFrame. For speed, we will only consider documents with the following 8 categories. 

In [29]:
from sklearn.datasets import fetch_20newsgroups

In [30]:
cats = [
    "rec.sport.hockey",
    "rec.sport.baseball",
    "soc.religion.christian",
    "alt.atheism",
    "comp.graphics",
    "comp.windows.x",
    "talk.politics.mideast",
    "talk.politics.guns",
]  # We'll only consider these categories out of 20 categories for speed.

newsgroups_train = fetch_20newsgroups(
    subset="train", remove=("headers", "footers", "quotes"), categories=cats
)
X_news_train, y_news_train = newsgroups_train.data, newsgroups_train.target
df = pd.DataFrame(X_news_train, columns=["text"])
df["target"] = y_news_train
df["target_name"] = [
    newsgroups_train.target_names[target] for target in newsgroups_train.target
]
df

,text,target,target_name
0,"You know, I was reading 18 U.S.C. 922 and some...",6,talk.politics.guns
1,\n\n\nIt's not a bad question: I don't have an...,1,comp.graphics
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics
3,"The following problem is really bugging me,\na...",2,comp.windows.x
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast
...,...,...,...
4558,Hi Everyone ::\n\nI am looking for some soft...,1,comp.graphics
4559,Archive-name: x-faq/part3\nLast-modified: 1993...,2,comp.windows.x
4560,"\nThat's nice, but it doesn't answer the quest...",6,talk.politics.guns
4561,"Hi,\n I just got myself a Gateway 4DX-33V ...",2,comp.windows.x


In [31]:
newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.windows.x',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast']

<br><br>

<!-- BEGIN QUESTION -->

### 2.1 Preprocessing using [spaCy](https://spacy.io/)
rubric={points:8}

In this exercise you'll prepare the data for topic modeling using [spaCy](https://spacy.io/). Preprocessing is a crucial step before training an LDA model and it markedly affects topic modeling results. So let's carry out preprocessing. 

**Your tasks:** 

Write code to carry out preprocessing of the "text" column in the dataframe above and store the preprocessed text in a new column called "text_pp" in the dataframe. 


Note that there is no such thing as "perfect" preprocessing. You'll have to make your own judgments and decisions on which tokens are more informative and which ones are less informative for the given task. Some common text preprocessing steps for topic modeling are: 
- getting rid of slashes or other weird characters
- sentence segmentation and tokenization      
- getting rid of urls and email addresses
- getting rid of other fairly unique tokens which are not going to help us in topic modeling  
- excluding stopwords and punctuation 
- lemmatization        

You might have to go back and forth between the preprocessing and topic modeling and interpretation steps in the next exercises. 

> Check out [these available attributes](https://spacy.io/api/token#attributes) for `token` in spaCy which might help you with preprocessing. 

> You can also get rid of words with specific POS tags. [Here](https://spacy.io/api/annotation/#pos-en) is the list of part-of-speech tags used in spaCy. 

> Note that preprocessing the corpus might take some time. So here are a couple of suggestions: 1) During the debugging phase, work on a smaller subset of the data. 2) Once you're done with the preprocessing part, you might want to save the preprocessed data so that you don't run the preprocessing part every time you run the notebook. 


<div class="alert alert-warning">

Solution_2_1
    
</div>

_Type your answer here, replacing this text._

In [32]:
from spacy import load
from sklearn.base import BaseEstimator, TransformerMixin

nlp = load('en_core_web_md')

class Embedder(BaseEstimator, TransformerMixin):
    def __init__(self, embedding):
        self.embedding = embedding
        
    def fit(self, X, y = None):
        return self
    
    def transform(self, X, y = None):
        return np.vstack([text.vector for text in self.embedding.pipe(X)])

In [33]:
def preprocess(doc, 
               min_token = 2, 
               rm_pos = ['SPACE', 'PUNCT', 'PRON', 'ADV', 'CCONJ', 'PART', 'CCONJ', 'INTJ', 'DET', 'ADP', 'SCONJ', 'AUX', 'X']):
    clean_text = []
    for token in doc:
        if (
            (token.is_stop is False) 
            and (len(token) > min_token)
            and (token.pos_ not in rm_pos)
            and (not token.like_email)
            and (not token.like_url)
        ):
            clean_text.append(token.lemma_.lower())
    return ' '.join(clean_text)

In [34]:
df['text_pp'] = [preprocess(text) for text in nlp.pipe(df['text'])]
df.head()

,text,target,target_name,text_pp
0,"You know, I was reading 18 U.S.C. 922 and some...",6,talk.politics.guns,know read u.s.c. 922 sence wonder help u.s.c. ...
1,\n\n\nIt's not a bad question: I don't have an...,1,comp.graphics,bad question ref list algorithm think bit hard...
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics,actuallay hand support idea have newsgroup asp...
3,"The following problem is really bugging me,\na...",2,comp.windows.x,following problem bug appreciate help create w...
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast,late upi foreign ministry spokesman ferhat ata...


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.2 Justification
rubric={points:2}

**Your tasks:**

Outline the preprocessing steps you carried out in the previous exercise and provide a brief justification for these steps. 

<div class="alert alert-warning">

Solution_2_2
    
</div>

1. Removing all stops works since they are unlikely have much discriminatory power between topics.
2. Remove any token that may be more than 1 in length. There may be tokens of length greater than 1 that may be sparse or too specific to certain topics.
3. Remove tokens that have POS labels list in the `preprocess` function. These are spaces, unclassified, or parts of speech that may not carry much discrimination power in my interpretation of the topics.
4. Removing emails since they are likely to be unique tokens. However, in separate iteration of the model, preserving the domain or top-level domain may give more discrimination power (ie. ".gov")
5. Removing URLs for the same reason as 4.
6. Words are lemmatized since a word can be vary in its form depending on the context of speech. However, for this analysis we are not concerned with the variants of a word, but the its base form. This way we can ensure that the same word, in different forms, are aggregated.
7. Lower case all tokens since a computer will be unable to infer upper cased letters in a word does not change the worlds meaning, at least for the type of text I believe is present in this corpus.

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.3 Build a topic model using sklearn's LatentDirichletAllocation
rubric={points:4}

**Your tasks:**
1. Create a topic model on the preprocessed data using [sklearn's `LatentDirichletAllocation`](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html). Pick a reasonable number for `n_components`, i.e., number of topics and briefly justify your choice.

<div class="alert alert-warning">

Solution_2_3
    
</div>

In [35]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.compose import make_column_transformer

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['text_pp'])

# Topics
# . Religion: Christian, Atheism
# . Computation: Graphics, Windows
# . Sports: Baseball, Hockey,
# . Politics: Guns, Middle East
n_topics = 4
lda = LatentDirichletAllocation(n_components = n_topics, 
                                learning_method = 'batch', 
                                max_iter = 10, 
                                random_state = 123)
document_topics = lda.fit_transform(X)

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.4 Exploring word topic association
rubric={points:5}

**Your tasks:**
1. Show top 10 words for each of your topics and suggest labels for each of the topics (similar to how we came up with labels "health and nutrition", "fashion", and "machine learning" in the toy example we saw in class). 

> If your topics do not make much sense, you might have to go back to preprocessing in Exercise 2.1, improve it, and train your LDA model again. 

<div class="alert alert-warning">

Solution_2_4
    
</div>

In [36]:
sorting = np.argsort(lda.components_, axis=1)[:, ::-1]
feature_names = np.array(vectorizer.get_feature_names_out())

In [37]:
import mglearn

mglearn.tools.print_topics(
    topics=range(n_topics),
    feature_names=feature_names,
    sorting=sorting,
    topics_per_chunk=5,
    n_words=10,
)

topic 0       topic 1       topic 2       topic 3       
--------      --------      --------      --------      
god           file          gun           game          
people        program       people        team          
know          use           think         play          
think         image         know          year          
believe       window        say           player        
say           available     israel        go            
armenian      include       time          say           
time          run           go            season        
jesus         entry         thing         win           
turkish       server        good          good          




<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.5 Exploring document topic association
rubric={points:5}

**Your tasks:**
1. Show the document topic assignment of the first five documents from `df`. 
2. Comment on the document topic assignment of the model. 

<div class="alert alert-warning">

Solution_2_5
    
</div>

Below is a data frame containing assignments of preprocessed text using the Latent Dirichlet Allocation algorithm. The numeric values communicate the degree in which a topic is "present" in a given document.

From the previous section top 10 words for each topic, I interpret each topic to be:
- topic 0: Religion
- topic 1: Computational
- topic 2: Politics
- topic 3: Sports

In [38]:
(
    pd.concat([df, pd.DataFrame(np.round(document_topics, 4) * 100)], 
              axis = 1)
    .iloc[:, 3:]
    .head(n = 5)
)

,text_pp,0,1,2,3
0,know read u.s.c. 922 sence wonder help u.s.c. ...,24.86,0.44,74.25,0.44
1,bad question ref list algorithm think bit hard...,0.37,98.91,0.37,0.36
2,actuallay hand support idea have newsgroup asp...,0.46,31.89,67.21,0.44
3,following problem bug appreciate help create w...,0.98,97.14,0.99,0.89
4,late upi foreign ministry spokesman ferhat ata...,64.81,0.60,34.00,0.59


**The first document is predominantly characterized as the third topic, interpreted as "Politics". There is also some component that aligns with the first topic, "Religion". I do believe that the topic model applied was accurate in grouping this document into "Politics". I am uncertain as to where the "Religion" came from.**

In [39]:
print(df.loc[0, 'text'])

You know, I was reading 18 U.S.C. 922 and something just did not make 
sence and I was wondering if someone could help me out.

Say U.S.C. 922 :

(1) Except as provided in paragraph (2), it shall be unlawful for
any person to transfer or possess a machinegun.

 Well I got to looking in my law dictionary and I found that a "person" 
might also be an artificial entity that is created by government 
and has no rights under the federal constitution. So, what I 
don't understand is how a statute like 922 can be enforced on 
an individual. So someone tell me how my government can tell
me what I can or cannot possess. Just passing a law 
does not make it LAW. Everyone knows that laws are constitional
until it goes to court. So, has it ever gone to court, not
just your run of the mill "Ok I had it I am guilty, put me in jail"

Has anyone ever claimed that they had a right to possess and was told
by the Supreme Court that they didn't have that right?





**The second document was strongly assigned to the second topic, which aligns well with "Computational" given the words present in the document (ex. algorithm, bisector, radius).**

In [40]:
print(df.loc[1, 'text'])




It's not a bad question: I don't have any refs that list this algorithm
either. But thinking about it a bit, it shouldn't be too hard.

1) Take three of the points and find the plane they define as well as
the circle that they lie on (you say you have this algorithm already)

2) Find the center  of this circle. The line passing through this center
perpendicular to the plane of the three points passes through the center of
the sphere.

3) Repeat with the unused point and two of the original points. This
gives you two different lines that both pass through the sphere's
origin. Their interection is the center of the sphere.

4) the radius is easy to compute, it's just the distance from the center to
any of the original points.

I'll leave the math to you, but this is a workable algorithm. :-)


An alternate method would be to take pairs of points: the plane formed
by the perpendicular bisector of each line segment pair also contains the
center of the sphere. Three pairs will form three

**The topic modeling missed the mark with the third document as it seems it would be better served in the "Computational" group, based on words like "graphic", "programming", etc. "Computational" was present as one of the predominant topics, but "Politics" took preference**

In [41]:
print(df.loc[2, 'text'])


Actuallay I don't, but on the other hand I don't support the idea of having
one newsgroup for every aspect of graphics programming as proposed by Brian,
in his reply to my original posting.
I would suggest a looser structure more like a comp.graphics.programmer,
comp.graphics.hw_and_sw
The reason for making as few groups as possible is for the same reason you
say we shouldn't spilt up, not to get to few postings every day.
I takes to much time to browse through all postings just to find two or 
three I'm interested in.

I understand and agree when you say you want all aspects of graphics in one
meeting. I agree to some extension. I see news as a forum to exchange ideas,
help others or to be helped. I think this is difficult to achive if there
are so many different things in one meeting.

Good evening netters|-)


**The topic modeling allocated this document accurately in the "Computational" group.**

In [42]:
print(df.loc[3, 'text'])

The following problem is really bugging me,
and I would appreciate any help.

I create two windows:

w1 (child to root) with event_mask = ButtonPressMask|KeyPressMask;
w2 (child to w1) with do_not_propagate_mask = ButtonPressMask|KeyPressMask;


Keypress events in w2 are discarded, but ButtonPress events fall through
to w1, with subwindow set to w2.

FYI, I'm using xnews/olvwm.

Am I doing something fundamentally wrong here?


**Topic modeling places this in Topic 0, which was attributed to "Religion". However, the document appears more in like with politics.**

In [43]:
print(df.loc[4, 'text'])



  This is the latest from UPI 

     Foreign Ministry spokesman Ferhat Ataman told journalists Turkey was
     closing its air space to all flights to and from Armenia and would
     prevent humanitarian aid from reaching the republic overland across
     Turkish territory.

  
   Historically even the most uncivilized of peoples have exhibited 
   signs of compassion by allowing humanitarian aid to reach civilian
   populations. Even the Nazis did this much.

   It seems as though from now on Turkey will publicly pronounce 
   themselves 'hypocrites' should they choose to continue their
   condemnation of the Serbians.



--


<!-- END QUESTION -->

<br><br><br><br>

<!-- BEGIN QUESTION -->

## Exercise 3: Short answer questions 
<hr>

rubric={points:6}

1. In lecture 19, we talked about multi-class classification. Comment on how each model in the list below might be handling multiclass classification. Check `scikit-learn` documentation for each of these models when you answer this question.  
    - Decision Tree
    - KNN
    - Random Forest    
    - Logistic Regression
    - SVM RBF
2. What is transfer learning in natural language processing or computer vision? Briefly explain.     
3. In Lecture 19 we briefly discussed how neural networks are sort of like `Pipeline`s, in the sense that they involve multiple sequential transformations of the data, finally resulting in the prediction. Why was this property useful when it came to transfer learning?

<div class="alert alert-warning">

Solution_3
    
</div>

**Q1**
- Decision Tree: Outputs the class that contains the greatest purity. A leaf node outputs only one class. Leaf node output value could be one of many possible values, not restricted to one of two.
- KNN: Based on similarity. There classes should also have minimal overlap for good discrimination.
- RandomForests: It is a vote of multiple Decision Trees with the majority winning.
- Logistic Regression: Creates a vector of coefficients for each class. The dot product between the coefficient vector and the data matrix gives the probability. The highest probability from the set of equations is selected.
- SVM RBF: Uses a one-vs-one approach.

**Q2**

Transfer learning is using a pre-trained model as a starting point. 

For computer vision, or image classification, using pixel values as input for simpler machine learning algorithms is not adequate. A pre-trained neural net can then be used as a feature extraction step to convert the image pixels into, potentially, more meaningful features that can be used for classification with rudimentary algorithms.

Similarly, with natural language processing, pre-train neural nets are used to map words into a vector space based on their similarity to words that the neural net was trained on. This may provide greater context, or semantic meaning, that would otherwise be missed used conventional count vectorization.

**Q3**

Given that it is a pipeline, we are able to tap into different layers of the neural net and use a layer of our choosing for our own modeling. Once more, tapping into a layer in a neural net may be akin to a complex feature extraction method. 

A neural net can be used to extract features from, for example, an image which we can later used for our own models. The neural net itself may not be adequate for the classification since it may not have adequate labels for our purposes. We can then train a simpler model, using a the features from a layer in the neural net, as the input.

<!-- END QUESTION -->

<br><br><br><br>

**PLEASE READ BEFORE YOU SUBMIT:** 

When you are ready to submit your assignment do the following:

1. Run all cells in your notebook to make sure there are no errors by doing `Kernel -> Restart Kernel and Clear All Outputs` and then `Run -> Run All Cells`. 
2. Notebooks with cell execution numbers out of order or not starting from "1" will have marks deducted. Notebooks without the output displayed may not be graded at all (because we need to see the output in order to grade your work).
3. Upload the assignment using Gradescope's drag and drop tool. Check out this [Gradescope Student Guide](https://lthub.ubc.ca/guides/gradescope-student-guide/) if you need help with Gradescope submission. 
4. Make sure that the plots and output are rendered properly in your submitted file. If the .ipynb file is too big and doesn't render on Gradescope, also upload a pdf or html in addition to the .ipynb so that the TAs can view your submission on Gradescope. 

![](img/eva-well-done.png)